In [2]:
from tweetnlp import Task, data, eval, classify

# Simple scoring

In [3]:
# Using the test labels as predictions should return a perfect score
task = Task.emoji
pred = data.test_labels(task)
s = eval.score(pred, task)
s == 1.0

True

# Evaluating Cardiff models

In [4]:
# Some Cardiff models on huggingface return human-readable labels and some integer ids
# So we cannot yet elegantly decide against which labels to compare...
task = Task.emotion
clf = classify.PretrainedCardiffClassifier(task)
s = eval.evaluate(clf, task, preproc=False)
s

Loading model cardiffnlp/twitter-roberta-base-emotion


Preprocessing: 100%|██████████| 1421/1421 [00:00<00:00, 193906.56it/s]
Classifying: 1421it [01:13, 19.38it/s]

First 10 labels: ['sadness', 'anger', 'sadness', 'optimism', 'optimism', 'joy', 'sadness', 'sadness', 'optimism', 'joy']


0.0